In [ ]:
import inspect

In [ ]:
import numpy as np
import math
import pandas as pd

In [ ]:
import sample_functions

In [ ]:
#def toBin (val, size):
#    return list(reversed(list(map(lambda x: int(x),'{:0{}b}'.format(val, size)))))
from dt2code import toBin

In [ ]:
def log2_12_12(a11, a10, a9, a8, a7, a6, a5, a4, a3, a2, a1, a0):
    bits = [a11, a10, a9, a8, a7, a6, a5, a4, a3, a2, a1, a0]
    
    def f(i): return min((2**12)-1,round(math.log2((i))*(((2**12)-1)/12))) if i != 0 else 0

    v1 = 0
    for i, b in enumerate(reversed(bits)):
        v1 += b << i
    mathval1 = f(v1)
    # Output as list of bits (LSB first)
    return toBin(mathval1, 12)


In [ ]:
#def deduce_fonction_prototype(fonction):
#    nb_input = len (inspect.signature(fonction).parameters)
#    fake_parametres = [0] * nb_input
#    nb_output = len(fonction(*fake_parametres))
#    return nb_input, nb_output
from dt2code import deduce_fonction_prototype
NBITS_INPUT, NBITS_OUTPUT = deduce_fonction_prototype(log2_12_12)

In [ ]:
print (NBITS_INPUT, NBITS_OUTPUT)

In [ ]:
def build_composed_training_samples():
    data = []
    for v1 in range(2**NBITS_INPUT):
        binv1 = sample_functions.toBin(v1, NBITS_INPUT)
        # log2_12_12 expects MSB first
        out_bits = log2_12_12(*reversed(binv1))
        row = {}
        for i in range(NBITS_INPUT):
            row[f'a{i}'] = binv1[i]
        for i in range(NBITS_INPUT,NBITS_INPUT+6):
            row[f'a{i}'] = out_bits[i-6]
        for i in range(NBITS_OUTPUT):
            row[f'r{i}'] = out_bits[i]
        data.append(row)
    df = pd.DataFrame(data)
    return df


In [ ]:
theDataframe = build_composed_training_samples()
theDataframe.head()
theDataframe.to_excel("output.xlsx")

In [ ]:
def build_X_y(idxOfBitToEncode, hypothesis, df=None):
    if df is None:
        raise ValueError("A DataFrame of training samples must be provided.")
    # Build filter mask
    mask = pd.Series([True] * len(df))
    for hyp in hypothesis:
        mask &= (df[f'a{hyp[0]}'] == hyp[1])
    filtered = df[mask]
    X = filtered[[f'a{i}' for i in range(NBITS_INPUT+6)]].values.tolist()
    y = filtered[f'r{idxOfBitToEncode}'].values.tolist()
    return X, y




In [ ]:
# X, y = build_X_y(5, hypothesis=[], df=theDataframe)
# from sklearn.tree import DecisionTreeClassifier
# tree = DecisionTreeClassifier().fit(X, y)
# X[0]


In [ ]:
# from sklearn.tree import export_text
# print(export_text(tree))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
def full_composed_abstract_tree   (listOfIdxOfBitToEncode,  hypothesis):

    idxOfBitToEncode = listOfIdxOfBitToEncode[0]

    X, y = build_X_y( idxOfBitToEncode, hypothesis, theDataframe)

    if len(y) == 1:
        if (y[0] == 0):
            result = {'value': 'pass'}
        else: 
            result = {'value': f"r{idxOfBitToEncode} = {y[0]}"}
        if (len(listOfIdxOfBitToEncode) != 1):
            listOfIdxOfBitToEncode.pop(0)
            result['subtree'] = full_composed_abstract_tree(listOfIdxOfBitToEncode, hypothesis)
        return result
    
    tree = DecisionTreeClassifier().fit(X, y) # max_depth=2
    # display_tree(tree)
    node_id=0

    if tree.tree_.children_left[node_id] == tree.tree_.children_right[node_id]:  # Leaf node
        listOfIdxOfBitToEncode.pop(0)  
        if len(listOfIdxOfBitToEncode) != 0:
            if (y[0] == 0):
                result = {'value': 'pass'}
            else:
                result = {'value': f"r{idxOfBitToEncode} = {y[0]}"}
            result['subtree']= full_composed_abstract_tree(listOfIdxOfBitToEncode, hypothesis)

            # listOfIdxOfBitToEncode.insert(0, sav)  # Restore the index for the next call
            return result
        else:
            if (y[0] == 0):
                result = {'value': 'pass'}
            else:
                result = {'value': f"r{idxOfBitToEncode} = {y[0]}"}
            # listOfIdxOfBitToEncode.insert(0, sav)
            return result
    else:
        result = {
            'feature': tree.tree_.feature[node_id],
            # 'threshold': "", #tree.tree_.threshold[node_id], #
            # 'value': "", # tree.tree_.value[node_id][0].tolist(), # 
        }
        savl = listOfIdxOfBitToEncode.copy()
        result['left'] = full_composed_abstract_tree(listOfIdxOfBitToEncode, hypothesis + [(tree.tree_.feature[node_id], 0)])
        listOfIdxOfBitToEncode = savl.copy()
        result['right'] = full_composed_abstract_tree(listOfIdxOfBitToEncode, hypothesis + [(tree.tree_.feature[node_id], 1)])
        listOfIdxOfBitToEncode = savl.copy()

        return result



In [ ]:
listOfIdxOfBitToEncode = [5, 4, 3, 2]#list(reversed(range(6)))
hypothesis = []
abstree = full_composed_abstract_tree(listOfIdxOfBitToEncode, hypothesis)

In [ ]:
from dt2code import convert_numpy_types, remove_pass_and_empty
import json
abstree_serializable = convert_numpy_types(abstree)
light_abstree = remove_pass_and_empty(abstree_serializable)
# print(json.dumps(light_abstree, indent=2))

In [ ]:
from dt2code import build_abstree_to_json
build_abstree_to_json(light_abstree, 'abstree.json')

In [ ]:
from dt2code import generate_function_code, save_function_to_file, read_abstree_from_json
function_code = generate_function_code(read_abstree_from_json('abstree.json'), 0, NBITS_INPUT+6, NBITS_OUTPUT)
save_function_to_file(function_code, 'abstree.py')

In [ ]:
from dt2code import generate_function_asm_code

with open ("brute_code/fonction.s", "w") as ficout:
    for line in generate_function_asm_code(read_abstree_from_json('abstree.json')):
        # print (line)
        ficout.write(line+"\n")


In [ ]:
from dt2code import abstree_to_asm6502_code
with open ("brute_code/function_core.s", "w") as ficout:
    for line in abstree_to_asm6502_code(read_abstree_from_json('abstree.json'), indent=1):
        # print (line)
        ficout.write(line+"\n")

In [ ]:
from dt2code import optimize_ASM_code
optimize_ASM_code("brute_code/function_core.s")